# IBM Data Science Coursera Capstone Project

## Battle of the Neighborhoods - Toronto Cuisine

### Introduction

This jupyter notebook will be used for the Applied Data Science Coursera Capstone Project. 

A client is looking to open a restaurant in Toronto and is interested in which location they should open it. Because of the ethnic diversity, Torontos’s restaurant industry is thriving. Many other factors contributing to the restaurant scene including the ethnic diversity and large population. The large population of Toronto provides restaurants with a large customer base and allows niche restaurants to receive support from the population. The restaurant scene is quite competitive  and restaurants of lower quality often go out of the business leaving higher quality. 

Toronto has a myriad of neighborhoods and suburbs where our client can open up their restaurant. Our client is a national company looking to expand their restaurant reach into Toronto with of opening up multiple restaurants in Toronto. They have come to us to provide a recommendation on which neighborhood they should open up their first Italian restaurant in.


### Data

For this project, I will be utilizing the Foursquare location data to provide information on neighborhoods, venues, restaurant ratings and location data. We can utilize the Foursquare data to pin point areas or clusters with restaurants with large engagement, high ratings and reviews. The Foursquare data can also be used to understand density of population of the neighborhoods and the restaurants within them. For example, 

In addition to the Foursquare data, I will also be looking at Toronto neighborhood data. These two data sets can be used in conjunction to pinpoint which neighborhood certain restaurants are located in. This can help us determine which neighborhood quality restaurants are located in and can help us pinpoint a zip code for the new restaurant. 

I will use all the data to find restaurants with a lot of reviews and high ratings. These restaurants will be mapped on folium and be grouped by zip code and neighborhoods. The neighborhoods with greater density of these clusters can show us potential locations for this new restaurant. If our client isn’t looking to build a location in a neighborhood dense with restaurants rife with competition, we can then look at neighborhood clusters that are less dense for other alternatives on where to build the new restaurant location.


### Methodology

In [2]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

     |████████████████████████████████| 79 kB 7.6 MB/s  eta 0:00:01
  Created wheel for folium: filename=folium-0.5.0-py3-none-any.whl size=76240 sha256=d108c1f4671ed8c28875313866267cdf484d31f94b8b28013739d7273a592b7b
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/b2/2f/2c/109e446b990d663ea5ce9b078b5e7c1a9c45cca91f377080f8
Successfully built folium
Folium installed
Libraries imported.


In [324]:
CLIENT_ID = '' # Foursquare ID
CLIENT_SECRET = '' # Foursquare Secret
ACCESS_TOKEN = '' # FourSquare Access Token
VERSION = '20180604'
LIMIT = 100
RADIUS = 10000
search_query = 'restaurant'


In [325]:
from bs4 import BeautifulSoup
url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(url, "html.parser")

table_contents = []
table = soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['Postal Code'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)
        
# print(table_contents)
df = pd.DataFrame(table_contents)
df['Borough'] = df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

df.head(11)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


### Get Latitude and Longitude Coordinates of Neighborhoods

In [326]:
import io

geo_url = "http://cocl.us/Geospatial_data"
geo_coord = requests.get(geo_url).text
df_geo_coord = pd.read_csv(io.StringIO(geo_coord))
df_geo_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [327]:
#Merge Latitude and Longitude with Toronto Postal Code Dataframe

df_toronto = pd.merge(df, df_geo_coord, on='Postal Code')
df_toronto.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [328]:
#Latitude and Longitude coordinators of Toronto

address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### Map of Neighborhoods in Toronto

In [329]:
import folium

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Explore Neighborhoods in Toronto

In [330]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [331]:
#specifying that we want to explore venues and neighborhoods in the Toronto
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )
print(toronto_venues.shape)
toronto_venues.head()

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills North
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview East
The Danforth

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
2,Parkwoods,43.753259,-79.329656,Tim Hortons,43.760668,-79.326368,Café
3,Parkwoods,43.753259,-79.329656,A&W,43.760643,-79.326865,Fast Food Restaurant
4,Parkwoods,43.753259,-79.329656,Bruno's valu-mart,43.746143,-79.324630,Grocery Store


#### How many venues were returned for each neighborhood?

In [332]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,46,46,46,46,46,46
"Alderwood, Long Branch",27,27,27,27,27,27
"Bathurst Manor, Wilson Heights, Downsview North",31,31,31,31,31,31
Bayview Village,17,17,17,17,17,17
"Bedford Park, Lawrence Manor East",39,39,39,39,39,39
Berczy Park,100,100,100,100,100,100
"Birch Cliff, Cliffside West",15,15,15,15,15,15
"Brockton, Parkdale Village, Exhibition Place",100,100,100,100,100,100
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",15,15,15,15,15,15


In [333]:
#Unique venue categories in Toronto 
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 327 uniques categories.


In [334]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Zoo,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Carpet Store,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Newsagent,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Lab,Photography Studio,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,Soup Place,South American Restaur

In [335]:
#group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Zoo,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Carpet Store,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsagent,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Lab,Photography Studio,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,Soup Place,South American Restaur

In [336]:
#frequency of most common venues

num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                  venue  freq
0    Chinese Restaurant  0.15
1         Shopping Mall  0.09
2                Bakery  0.04
3  Caribbean Restaurant  0.04
4           Coffee Shop  0.04


----Alderwood, Long Branch----
               venue  freq
0     Discount Store  0.11
1        Pizza Place  0.07
2           Pharmacy  0.07
3               Park  0.07
4  Convenience Store  0.04


----Bathurst Manor, Wilson Heights, Downsview North----
               venue  freq
0        Pizza Place  0.06
1        Coffee Shop  0.06
2               Park  0.06
3               Bank  0.06
4  Convenience Store  0.03


----Bayview Village----
                 venue  freq
0                 Bank  0.12
1        Grocery Store  0.12
2          Gas Station  0.12
3         Intersection  0.12
4  Japanese Restaurant  0.12


----Bedford Park, Lawrence Manor East----
                  venue  freq
0           Coffee Shop  0.08
1    Italian Restaurant  0.08
2                  Bank  0.05
3        Sandwich Place

                  venue  freq
0        Clothing Store  0.10
1           Coffee Shop  0.06
2  Fast Food Restaurant  0.06
3            Restaurant  0.06
4          Dessert Shop  0.04


----Lawrence Park----
                  venue  freq
0                  Café  0.22
1          College Quad  0.11
2           College Gym  0.11
3  Gym / Fitness Center  0.11
4             Bookstore  0.11


----Leaside----
                    venue  freq
0     Sporting Goods Shop  0.07
1             Coffee Shop  0.07
2       Electronics Store  0.05
3           Grocery Store  0.05
4  Furniture / Home Store  0.05


----Little Portugal, Trinity----
                venue  freq
0                Café  0.08
1                 Bar  0.07
2          Restaurant  0.06
3  Italian Restaurant  0.04
4              Bakery  0.04


----Malvern, Rouge----
                  venue  freq
0  Fast Food Restaurant  0.10
1           Coffee Shop  0.10
2                 Trail  0.10
3                Bakery  0.05
4   Arts & Crafts Store  0.0

In [370]:
#function to return most common venues in descending order

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [371]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Shopping Mall,Bakery,Coffee Shop,Sandwich Place,Caribbean Restaurant,Japanese Restaurant,Sri Lankan Restaurant,Bank,Restaurant
1,"Alderwood, Long Branch",Discount Store,Pizza Place,Pharmacy,Park,Gas Station,Skating Rink,Sandwich Place,Donut Shop,Garden Center,Bagel Shop
2,"Bathurst Manor, Wilson Heights, Downsview North",Pizza Place,Park,Coffee Shop,Bank,Mediterranean Restaurant,Ski Chalet,Fried Chicken Joint,Sushi Restaurant,Supermarket,Men's Store
3,Bayview Village,Intersection,Bank,Japanese Restaurant,Grocery Store,Gas Station,Trail,Skating Rink,Chinese Restaurant,Restaurant,Café
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Fast Food Restaurant,Bank,Sandwich Place,Juice Bar,Baby Store,Pub,Bagel Shop,Bakery


In [372]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 10

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 5, 5, 5, 4, 4, 5, 4, 4, 5], dtype=int32)

In [373]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,5.0,Park,Pharmacy,Bus Stop,Shopping Mall,Fish & Chips Shop,Shop & Service,Supermarket,Food & Drink Shop,Fast Food Restaurant,Skating Rink
1,M4A,North York,Victoria Village,43.725882,-79.315572,6.0,Coffee Shop,Sporting Goods Shop,Gym / Fitness Center,Pizza Place,Men's Store,French Restaurant,Golf Course,Portuguese Restaurant,Hockey Arena,Health & Beauty Service
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,4.0,Coffee Shop,Café,Park,Theater,Restaurant,Pub,Breakfast Spot,Bakery,Sushi Restaurant,Diner
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,4.0,Clothing Store,Coffee Shop,Fast Food Restaurant,Restaurant,Sushi Restaurant,Seafood Restaurant,Furniture / Home Store,Dessert Shop,Vietnamese Restaurant,Fried Chicken Joint
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,4.0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Park,Thai Restaurant,Café,Italian Restaurant,Ramen Restaurant,Yoga Studio,Bookstore


Let's look at the 10 clusters made to determine which clusters our client might like to open a restaurant. 

#### Cluster 0

In [358]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,0.0,Trail,Fast Food Restaurant,Coffee Shop,Spa,Construction & Landscaping,Martial Arts School,Supermarket,Caribbean Restaurant,Bank,Bakery
26,Scarborough,0.0,Bank,Gas Station,Coffee Shop,Bakery,Indian Restaurant,Yoga Studio,Thai Restaurant,Restaurant,Caribbean Restaurant,Athletics & Sports
34,North York,0.0,Coffee Shop,Restaurant,Pizza Place,Furniture / Home Store,Sandwich Place,Chinese Restaurant,Japanese Restaurant,Market,Bar,Massage Studio
44,Scarborough,0.0,Coffee Shop,Intersection,Bakery,Pizza Place,Trail,Fast Food Restaurant,Bus Line,Mexican Restaurant,Metro Station,Beer Store
49,North York,0.0,Coffee Shop,Intersection,Gas Station,Chinese Restaurant,Park,Athletics & Sports,Dim Sum Restaurant,Bakery,Convenience Store,Mediterranean Restaurant
56,York,0.0,Furniture / Home Store,Intersection,Grocery Store,Discount Store,Sandwich Place,Dessert Shop,Shopping Mall,Italian Restaurant,Gas Station,Fast Food Restaurant
65,Scarborough,0.0,Coffee Shop,Furniture / Home Store,Pharmacy,Restaurant,Electronics Store,Asian Restaurant,Intersection,Fast Food Restaurant,Indian Restaurant,Bakery
76,Mississauga,0.0,Coffee Shop,Hotel,Chinese Restaurant,Middle Eastern Restaurant,Fried Chicken Joint,Burrito Place,Bus Station,Bakery,Mexican Restaurant,Asian Restaurant
78,Scarborough,0.0,Chinese Restaurant,Shopping Mall,Bakery,Coffee Shop,Sandwich Place,Caribbean Restaurant,Japanese Restaurant,Sri Lankan Restaurant,Bank,Restaurant
82,Scarborough,0.0,Fast Food Restaurant,Convenience Store,Coffee Shop,Sandwich Place,Bank,Hobby Shop,Chinese Restaurant,Restaurant,Gas Station,Bakery


#### Cluster 1

In [359]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
46,North York,1.0,Park,Bank,Shopping Mall,Pizza Place,Moving Target,Vietnamese Restaurant,Grocery Store,Yoga Studio,Escape Room,Dry Cleaner
101,Etobicoke,1.0,Park,Bus Stop,Shopping Mall,Eastern European Restaurant,Ice Cream Shop,Italian Restaurant,Event Space,Dumpling Restaurant,Electronics Store,Escape Room


#### Cluster 2

In [360]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,North York,2.0,Park,Pool,Yoga Studio,Falafel Restaurant,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant


#### Cluster 3

In [361]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
53,North York,3.0,Vietnamese Restaurant,Baseball Field,Pharmacy,Yoga Studio,Event Space,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room


#### Cluster 4

In [374]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,4.0,Coffee Shop,Café,Park,Theater,Restaurant,Pub,Breakfast Spot,Bakery,Sushi Restaurant,Diner
3,North York,4.0,Clothing Store,Coffee Shop,Fast Food Restaurant,Restaurant,Sushi Restaurant,Seafood Restaurant,Furniture / Home Store,Dessert Shop,Vietnamese Restaurant,Fried Chicken Joint
4,Queen's Park,4.0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Park,Thai Restaurant,Café,Italian Restaurant,Ramen Restaurant,Yoga Studio,Bookstore
7,North York,4.0,Coffee Shop,Japanese Restaurant,Burger Joint,Pizza Place,Mobile Phone Shop,Breakfast Spot,Cafeteria,Café,Liquor Store,Asian Restaurant
9,Downtown Toronto,4.0,Coffee Shop,Gastropub,Japanese Restaurant,Theater,Bookstore,Italian Restaurant,Cosmetics Shop,Café,Clothing Store,Department Store
13,North York,4.0,Restaurant,Gym,Café,Bus Line,Supermarket,Sushi Restaurant,Coffee Shop,Salon / Barbershop,Clothing Store,Japanese Restaurant
15,Downtown Toronto,4.0,Coffee Shop,Café,Restaurant,Italian Restaurant,Cosmetics Shop,Bakery,Seafood Restaurant,Gastropub,Plaza,Clothing Store
19,East Toronto,4.0,Pub,Coffee Shop,Pizza Place,Breakfast Spot,Japanese Restaurant,Beach,Pharmacy,Bar,Bakery,Burger Joint
20,Downtown Toronto,4.0,Coffee Shop,Café,Hotel,Park,Japanese Restaurant,Restaurant,Art Gallery,Pharmacy,Cocktail Bar,Pub
23,East York,4.0,Coffee Shop,Sporting Goods Shop,Electronics Store,Furniture / Home Store,Grocery Store,Brewery,Sports Bar,Restaurant,Department Store,Bank


#### Cluster 5

In [375]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,5.0,Park,Pharmacy,Bus Stop,Shopping Mall,Fish & Chips Shop,Shop & Service,Supermarket,Food & Drink Shop,Fast Food Restaurant,Skating Rink
5,Etobicoke,5.0,Pharmacy,Bank,Grocery Store,Convenience Store,Playground,Shopping Mall,Golf Course,Café,Bakery,Park
11,Etobicoke,5.0,Park,Pizza Place,Hotel,Restaurant,Mexican Restaurant,Bank,Grocery Store,Gym,Clothing Store,Fish & Chips Shop
14,East York,5.0,Coffee Shop,Park,Pizza Place,Café,Sandwich Place,Skating Rink,Athletics & Sports,Liquor Store,Thai Restaurant,Beer Store
16,York,5.0,Pizza Place,Convenience Store,Park,Coffee Shop,Bank,Italian Restaurant,Gastropub,Bagel Shop,Field,Optical Shop
21,York,5.0,Pharmacy,Park,Pizza Place,Café,Discount Store,Grocery Store,Japanese Restaurant,Bus Stop,Food Truck,Coffee Shop
22,Scarborough,5.0,Park,Coffee Shop,Mobile Phone Shop,Chinese Restaurant,Fast Food Restaurant,Indian Restaurant,Pharmacy,Yoga Studio,Event Space,Dumpling Restaurant
27,North York,5.0,Pharmacy,Park,Coffee Shop,Pizza Place,Korean Restaurant,Bank,Bakery,Chinese Restaurant,Fast Food Restaurant,Tennis Court
28,North York,5.0,Pizza Place,Park,Coffee Shop,Bank,Mediterranean Restaurant,Ski Chalet,Fried Chicken Joint,Sushi Restaurant,Supermarket,Men's Store
39,North York,5.0,Intersection,Bank,Japanese Restaurant,Grocery Store,Gas Station,Trail,Skating Rink,Chinese Restaurant,Restaurant,Café


#### Cluster 6

In [376]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 6, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,6.0,Coffee Shop,Sporting Goods Shop,Gym / Fitness Center,Pizza Place,Men's Store,French Restaurant,Golf Course,Portuguese Restaurant,Hockey Arena,Health & Beauty Service
8,East York,6.0,Pizza Place,Brewery,Coffee Shop,Construction & Landscaping,Gym / Fitness Center,Flea Market,Fast Food Restaurant,Café,Gastropub,Bank
10,North York,6.0,Fast Food Restaurant,Grocery Store,Park,Pizza Place,Furniture / Home Store,Gas Station,Italian Restaurant,Coffee Shop,Fish Market,Mediterranean Restaurant
17,Etobicoke,6.0,Coffee Shop,Breakfast Spot,Shopping Mall,Fish & Chips Shop,Liquor Store,Shopping Plaza,Café,Electronics Store,Beer Store,Grocery Store
18,Scarborough,6.0,Pizza Place,Fast Food Restaurant,Bank,Pharmacy,Coffee Shop,Greek Restaurant,Filipino Restaurant,Sandwich Place,Liquor Store,Discount Store
29,East York,6.0,Coffee Shop,Indian Restaurant,Grocery Store,Brewery,Turkish Restaurant,Supermarket,BBQ Joint,Burger Joint,Shopping Mall,Afghan Restaurant
32,Scarborough,6.0,Ice Cream Shop,Intersection,Restaurant,Coffee Shop,Fast Food Restaurant,Grocery Store,Sandwich Place,Bowling Alley,Pizza Place,Convenience Store
38,Scarborough,6.0,Chinese Restaurant,Discount Store,Coffee Shop,Pizza Place,Grocery Store,Fast Food Restaurant,Rental Car Location,Bank,Sandwich Place,Asian Restaurant
51,Scarborough,6.0,Pizza Place,Ice Cream Shop,Beach,Restaurant,Sports Bar,Auto Garage,Pharmacy,Park,Ethiopian Restaurant,Dry Cleaner
60,North York,6.0,Hotel,Grocery Store,Pizza Place,Gas Station,Coffee Shop,Pharmacy,Fast Food Restaurant,Discount Store,Falafel Restaurant,Skating Rink


#### Cluster 7

In [377]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 7, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Scarborough,7.0,Breakfast Spot,Park,Playground,Burger Joint,Italian Restaurant,Falafel Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room


#### Cluster 8

In [378]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 7, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Scarborough,7.0,Breakfast Spot,Park,Playground,Burger Joint,Italian Restaurant,Falafel Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room


#### Cluster 9

In [379]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 7, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Scarborough,7.0,Breakfast Spot,Park,Playground,Burger Joint,Italian Restaurant,Falafel Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room


In [380]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Results and Discussion

Based on the clusters created, we can see a greater density of restaurants in clusters 0, 4, 5, and 6. We can see that in cluster 0, amongst the 1st most common venues, the most prevalent are restaurants and coffee shops. This is a good indicator that this area is bustling with traffic. When we further look into the 2nd most common venues for cluster 0, we see that intersections are common further enhancing our statement. Looking at cluster 4, we can see that amongst the 1st most common venues, coffee shops are prevalent while amongst the 2nd and 3rd most common venues are restaurants and cafes. The presence of many restaurants and cafes can give us some understanding this area must be competitive but that there is also a large population that is able to service these many restaurants. 

While it seems that cluster 5 has a lot of restaurants in the 4th, 5th and 6th most common venues, in the top most common venues we find pharmacies, pizza places, and general stores. If our client were looking to build a pizza restaurant, cluster 5 gives us an indication that the population in the area often services pizza places. In its top most common venues, cluster 6 seems to have a pizza places and fast food restaurants. If fast food was our clients interests, this would be a good location to look into. 

### Conclusion

From the data and results, we can recommend the client to look into building a restaurant in cluster 4 due to popularity and density of that area. If our client were looking to build a pizza restaurant, cluster 5 would be a good location to look into. 